![](../additional_materials/logos/darden_rice_logo_SM.png)

Code: *Anirudh Shah, Adam Zucker*

In [1]:
import pandas as pd

## Important candidates to track
- Ken Welch
- Robert Blackmon
- Pete Boland
- Wengay Newton

## Instructions
1. Download PDFs of relevant candidates' *Campaign Treasurer's Reports* from the [Easy Campaign Finance Portal](https://cityofstpetersburgfl.easyvotecampaignfinance.com/home/publicfilings), and save to `/campaign_finance_data/pdf`
2. Convert each candidate's PDF Campaign Treasurer's Report for the given month into a csv using [Tabula](https://tabula.technology/). Only include tables from pages with the header "Campaign Treasurer's Report - Itemized Contributions". Save the resulting csv to `/campaign_finance_data/raw`
3. Update the cell immediately below these instructions with the correct filepath to the relevant csv. Format should be `'./campaign_finance_data/raw/candidate_name_mm-dd-yy.csv'`
4. Update the file name in the final cell, `contrib_df.to_csv()`, to reflect the correct candidate and reporting date. Format should be `contrib_df.to_csv('./campaign_finance_data/processed/candidate_name_month_finance.csv', index = False)`.
5. Run the notebook!

In [2]:
# Import each candidate's most current Treasurer's Report after converting to csv using Tabula
contrib_df = pd.read_csv('./campaign_finance_data/raw/welch_08-20-21.csv')

# View original df formatting
contrib_df.head()

,(5)\rDate,"(7)\rFull Name\r(Last, Suffix, First, Middle)\rStreet Address &\rCity, State, Zip Code",(8)\rContributor\rTypeOccupation,(9)\rContribution\rType,(10)\rIn-kind\rDescription,(11)\rAmendment,(12)\rAmount,Unnamed: 7
0,(6)\rSequence\rNumber,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,08\r0921\r//,Franklin Knight\r400 66 Avenue S\rSt. Petersbu...,Individ,al,Check,NaN,NaN,$25.00
2,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,08\r1121\r//,"Mary Bailey\r685 59th Ave S St\rPetersburg, FL...",Individ,al,Check,NaN,NaN,$25.00
4,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# Remove all entries where the contribution amount (column 'Unnamed: 7') is null
contrib_df = contrib_df[contrib_df['Unnamed: 7'].notna()]

In [4]:
# Drop columns labeled '(11)\rAmendment' and '(12)\rAmount' columns
contrib_df = contrib_df.drop(axis=1, labels=['(11)\rAmendment', '(12)\rAmount'])

In [5]:
# Rename columns for legibility
contrib_df = contrib_df.rename(mapper={'(5)\rDate': 'Date', '(8)\rContributor\rTypeOccupation': 'Contributor Type', 
                                       '(9)\rContribution\rType': 'Occupation', '(10)\rIn-kind\rDescription': 'Payment Type', 
                                       'Unnamed: 7':'Amount', '(7)\rFull Name\r(Last, Suffix, First, Middle)\rStreet Address &\rCity, State, Zip Code':'Name'}, 
                                       axis=1)

In [6]:
contrib_df.head()

,Date,Name,Contributor Type,Occupation,Payment Type,Amount
1,08\r0921\r//,Franklin Knight\r400 66 Avenue S\rSt. Petersbu...,Individ,al,Check,$25.00
3,08\r1121\r//,"Mary Bailey\r685 59th Ave S St\rPetersburg, FL...",Individ,al,Check,$25.00
5,08\r1121\r//,Rahdert & Mortimer\rPLLC\r535 Central Avenue\r...,Busine,Law Firm\rs,Check,$250.00
7,08\r1121\r//,Pinellas Stonewall PAC\r2250 1st Ave N\rSt. Pe...,Politic,Political\rCommittee\rlCommittee,Check,$500.00
9,08\r1321\r//,Ric-Man Construction\rFlorida Inc\r3100 SW 15t...,Busine,Construction\rs,Check,"$1,000.00"


In [7]:
# This function reformats dates in 'Date' column as 'mm/dd/yy'
def parse_date(date_str):
    # 'slashes' determines where to place '/' in the 'date' string when concatenating
    slashes = [2, 5, 6]
    date = ""
    while len(date) < 8:
        for char in str(date_str):
            if char.isnumeric():
                date = date + char
                if len(date) in slashes:
                    date = date + '/'
    return date

In [8]:
parse_date('01\r1921\r//')

'01/19/21'

In [9]:
# contrib_df['Date'] = contrib_df['Date'].apply(parse_date)

In [10]:
contrib_df['Date'] = contrib_df['Date'].apply(lambda x: parse_date(x))

# # Converting dates to datetime objects
# contrib_df['Date'] = pd.to_datetime(contrib_df['Date'])

In [11]:
contrib_df.head()

,Date,Name,Contributor Type,Occupation,Payment Type,Amount
1,08/09/21,Franklin Knight\r400 66 Avenue S\rSt. Petersbu...,Individ,al,Check,$25.00
3,08/11/21,"Mary Bailey\r685 59th Ave S St\rPetersburg, FL...",Individ,al,Check,$25.00
5,08/11/21,Rahdert & Mortimer\rPLLC\r535 Central Avenue\r...,Busine,Law Firm\rs,Check,$250.00
7,08/11/21,Pinellas Stonewall PAC\r2250 1st Ave N\rSt. Pe...,Politic,Political\rCommittee\rlCommittee,Check,$500.00
9,08/13/21,Ric-Man Construction\rFlorida Inc\r3100 SW 15t...,Busine,Construction\rs,Check,"$1,000.00"


In [12]:
contrib_df['Name'] = contrib_df['Name'].apply(lambda x: x.split('\r'))

In [13]:
contrib_df['Full Name'] = contrib_df['Name'].apply(lambda x: x[0])

In [14]:
def extract_street_address(address_list):
    if len(address_list) > 3:
        del address_list[0]
        address = " ".join(address_list[:-1])
        return address
    else:
        return address_list[1]

In [15]:
contrib_df['Address'] = contrib_df['Name'].apply(lambda x: extract_street_address(x))


In [16]:
def split_city_state_zip(name_list):
#     print(name_list)

    val = name_list[-1]
    city = None
    state = None
    zipcode = None
    if val.isnumeric():
        zipcode = val
    else:
        city = val.split(',')[0]
        if len(val.split(',')) > 1:
            state_zip = val.split(',')[1].strip().split(' ')
            if len(state_zip) > 1:
                state = state_zip[0]
                zipcode = state_zip[1]
            else:
                state = state_zip[0]
    return city, state, zipcode

In [17]:
contrib_df['City'] = contrib_df['Name'].apply(lambda x: split_city_state_zip(x)[0])
contrib_df['State'] = contrib_df['Name'].apply(lambda x: split_city_state_zip(x)[1])
contrib_df['Zipcode'] = contrib_df['Name'].apply(lambda x: split_city_state_zip(x)[2])


In [18]:
contrib_df.head()

,Date,Name,Contributor Type,Occupation,Payment Type,Amount,Full Name,Address,City,State,Zipcode
1,08/09/21,"[Franklin Knight, 400 66 Avenue S, St. Petersb...",Individ,al,Check,$25.00,Franklin Knight,400 66 Avenue S,St. Petersburg,FL,33705
3,08/11/21,"[Mary Bailey, 685 59th Ave S St, Petersburg, F...",Individ,al,Check,$25.00,Mary Bailey,685 59th Ave S St,Petersburg,FL,
5,08/11/21,"[PLLC, 535 Central Avenue, St. Petersburg, FL ...",Busine,Law Firm\rs,Check,$250.00,Rahdert & Mortimer,PLLC 535 Central Avenue,St. Petersburg,FL,33701
7,08/11/21,"[Pinellas Stonewall PAC, 2250 1st Ave N, St. P...",Politic,Political\rCommittee\rlCommittee,Check,$500.00,Pinellas Stonewall PAC,2250 1st Ave N,St. Petersburg,FL,33713
9,08/13/21,"[Florida Inc, 3100 SW 15th St, Deerfield Beach...",Busine,Construction\rs,Check,"$1,000.00",Ric-Man Construction,Florida Inc 3100 SW 15th St,Deerfield Beach,FL,None


In [19]:
def parse_first_name(full_name):
    names = full_name.split(' ')
    titles = ['Mr', 'Mrs', 'Miss', 'Ms', 'Dr']
    if names[0] not in titles:
        first = names[0]
    else:
        first = names[1]

    res = ''.join([i for i in first if not i.isdigit()])
    return res

In [20]:
contrib_df['First'] = contrib_df['Full Name'].apply(lambda x: parse_first_name(x))
contrib_df['Last'] = contrib_df['Full Name'].apply(lambda x: x.split(' ')[-1].strip())

In [21]:
contrib_df.head()

,Date,Name,Contributor Type,Occupation,Payment Type,Amount,Full Name,Address,City,State,Zipcode,First,Last
1,08/09/21,"[Franklin Knight, 400 66 Avenue S, St. Petersb...",Individ,al,Check,$25.00,Franklin Knight,400 66 Avenue S,St. Petersburg,FL,33705,Franklin,Knight
3,08/11/21,"[Mary Bailey, 685 59th Ave S St, Petersburg, F...",Individ,al,Check,$25.00,Mary Bailey,685 59th Ave S St,Petersburg,FL,,Mary,Bailey
5,08/11/21,"[PLLC, 535 Central Avenue, St. Petersburg, FL ...",Busine,Law Firm\rs,Check,$250.00,Rahdert & Mortimer,PLLC 535 Central Avenue,St. Petersburg,FL,33701,Rahdert,Mortimer
7,08/11/21,"[Pinellas Stonewall PAC, 2250 1st Ave N, St. P...",Politic,Political\rCommittee\rlCommittee,Check,$500.00,Pinellas Stonewall PAC,2250 1st Ave N,St. Petersburg,FL,33713,Pinellas,PAC
9,08/13/21,"[Florida Inc, 3100 SW 15th St, Deerfield Beach...",Busine,Construction\rs,Check,"$1,000.00",Ric-Man Construction,Florida Inc 3100 SW 15th St,Deerfield Beach,FL,None,Ric-Man,Construction


In [22]:
contrib_df = contrib_df.drop(columns = ['Name', 'Full Name'])

In [23]:
contrib_df['Contributor Type'] = contrib_df['Contributor Type'].map({'Busine':'Business', 'Individ': 'Individual'})

In [24]:
def clean_occupation(occupation):
    if '\ral' in occupation:
        occupation = occupation.replace('\ral', '')
    occupation = occupation.replace('\r', ' ')
    if occupation == 'al':
        occupation = ''
    return occupation

In [25]:
contrib_df['Occupation'] = contrib_df['Occupation'].apply(lambda x: clean_occupation(x))

In [26]:
contrib_df = contrib_df[['Date', 'Amount', 'Payment Type', 'First', 'Last', 'Address', 'City', 'State', 'Zipcode', 'Occupation']]

In [27]:
contrib_df.head()

,Date,Amount,Payment Type,First,Last,Address,City,State,Zipcode,Occupation
1,08/09/21,$25.00,Check,Franklin,Knight,400 66 Avenue S,St. Petersburg,FL,33705,
3,08/11/21,$25.00,Check,Mary,Bailey,685 59th Ave S St,Petersburg,FL,,
5,08/11/21,$250.00,Check,Rahdert,Mortimer,PLLC 535 Central Avenue,St. Petersburg,FL,33701,Law Firm s
7,08/11/21,$500.00,Check,Pinellas,PAC,2250 1st Ave N,St. Petersburg,FL,33713,Political Committee lCommittee
9,08/13/21,"$1,000.00",Check,Ric-Man,Construction,Florida Inc 3100 SW 15th St,Deerfield Beach,FL,None,Construction s


In [28]:
contrib_df.dtypes

Date            object
Amount          object
Payment Type    object
First           object
Last            object
Address         object
City            object
State           object
Zipcode         object
Occupation      object
dtype: object

### Update the following cell per instructions

In [29]:
contrib_df.to_csv('./campaign_finance_data/processed/ken_welch_08-20-21_finance.csv', index=False)

---
## TO DO
- [ ] Remove salutations from names
- [ ] Weird names with numbers in the middle
- [ ] Accents
- [ ] Permute columns
- [ ] Recognize companies containing LLC, INC
- [ ] Date to datetime object -> group contributions by date

---
**BELOW:** If you need to concatenate more than one dataframe to comprise the month, use the code below. Replace file names where necessary.

In [30]:
# # Update this cell with filepaths to the relevant reports
# df1 = pd.read_csv('./campaign_finance_data/processed/ken_welch_juneH1_finance.csv')
# df2 = pd.read_csv('./campaign_finance_data/processed/ken_welch_juneH2_finance.csv')

In [31]:
# month_df = pd.concat([df1, df2], axis = 0)

# month_df.reset_index(drop = True, inplace = True)

In [32]:
# month_df.head(3)

In [33]:
# month_df.tail(3)

### Update the following cell per instructions

In [34]:
# month_df.to_csv('./campaign_finance_data/processed/ken_welch_june_finance.csv', index=False)